In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.applications import InceptionV3
import tensorflow as tf
import keras as keras
import pickle
import keras.backend as K
import cv2
import os
import shutil

## Splitting video into frames

In [ ]:
def get_frames(uploaded):
  if len(uploaded)==0:
    print("No file uploaded")
    return None
  else:
    filename = next(iter(uploaded))
    if os.path.isdir('frames'):
            shutil.rmtree('frames')
    os.makedirs('frames',exist_ok=True)
    cap = cv2.VideoCapture(str(filename))
    frame_count = 0
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame_count += 1
      frame_filename = f'frames/frame_{frame_count}.jpg'
      cv2.imwrite(frame_filename, frame)
    cap.release()
    frame_filenames = sorted(os.listdir('frames'))
    print(f"Extracted {frame_count} frames.")
    return frame_filenames

In [ ]:
def detect_all_objects(frame_filenames):
  if frame_filenames==None:
    print("No frames found")
    return None, None
  else:
    frame_obj_dict = {}
    for frame_filename in frame_filenames:
        frame_path = os.path.join('frames', frame_filename)
        img = tf.io.read_file(frame_path)
        img = tf.image.decode_image(img, channels=3)
        img = tf.cast(img, tf.float32)
        img = tf.image.resize(img, (299, 299))
        img = tf.keras.applications.inception_v3.preprocess_input(img)
        img_expanded = tf.expand_dims(img, axis=0)
        model = InceptionV3(weights='imagenet', include_top=True)
        prediction = model.predict(img_expanded,verbose=0)
        decoded_predictions = tf.keras.applications.inception_v3.decode_predictions(prediction, top=3)
        img_dict = {}
        for i, (imagenet_id, label, score) in enumerate(decoded_predictions[0]):
              img_dict[i+1] = label
        frame_obj_dict[frame_filename] = img_dict

    all_objects = []
    for frame_name in frame_obj_dict.keys():
      for obj_id in frame_obj_dict[frame_name].keys():
        all_objects.append(frame_obj_dict[frame_name][obj_id])
    all_obj = list(set(all_objects))

    return frame_obj_dict, all_obj

In [ ]:
def search_object(search_query,frame_obj_dict,all_obj):
  if frame_obj_dict==None:
    print("No frames found")
    return None
  else:
    obj_frames = []
    for frame_name in frame_obj_dict.keys():
      if search_query in frame_obj_dict[frame_name].values():
        obj_frames.append(frame_name)
    import matplotlib.pyplot as plt
    if len(obj_frames)==0:
      print("Object doesn't exist!!!")
      print("\nChoose from the list below")
      for obj in all_obj:
        print(f'\n {obj}')
      opt=input("What new object do you want to search for: ")
      obj_in_video(opt)
    else:
      for framee in obj_frames:
        frame_path = os.path.join('frames', framee)
        frame = cv2.imread(frame_path)
        print(f'\n{search_query} in {framee}')
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

In [ ]:
def obj_in_video(search_query):
  uploaded = files.upload()
  filenames = get_frames(uploaded)
  fram_obj_dict, objs = detect_all_objects(filenames)
  search_object(search_query,fram_obj_dict,objs)

In [ ]:
obj_in_video("book")